In [1]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Activation, Dense, BatchNormalization, concatenate, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape
from keras.callbacks import EarlyStopping
from keras.layers.core import SpatialDropout2D
from keras import backend as K
from tensorflow.keras.optimizers import Adam, RMSprop,SGD,Adagrad
import numpy as np
import pandas as pd
import glob
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
from keras.models import load_model
import itertools 
filterwarnings('ignore')
np.random.seed(101)
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Sat Jun 24 15:03:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.37       Driver Version: 527.37       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:06:00.0  On |                  N/A |
|  0%   51C    P8    21W / 215W |    459MiB /  8192MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
train_dir ="Dataset/wheat_leaf/Train"
val_dir="Dataset/wheat_leaf/Test"
test_dir="Dataset/wheat_leaf/Test"
labels = ['Healthy', 'septoria', 'stripe_rust']
label_np=np.array(labels)

In [4]:
import tensorflow as tf
import numpy as np
import os
import random


def set_seed(seed):
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)


seed = 101
set_seed(seed)

In [5]:
model1 = EfficientNetB7(include_top=False,input_shape=(224, 224,3), weights='imagenet')
input_shape = (224, 224)

In [6]:
datagen_train = ImageDataGenerator(rescale=1./255)

datagen_test = ImageDataGenerator(rescale=1./255)

datagen_val=ImageDataGenerator(rescale=1./255)

batch_size = 4
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,                                        
                                                    shuffle=True)

generator_val = datagen_val.flow_from_directory(directory=val_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)
generator_test=datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                 batch_size=batch_size,
                                                 shuffle=False)

Found 328 images belonging to 3 classes.
Found 79 images belonging to 3 classes.
Found 79 images belonging to 3 classes.


In [ ]:
headModel = model1.output
headModel = layers.Flatten()(headModel)
headModel = Dense(3, activation="softmax")(headModel)
model = Model(inputs=model1.input, outputs=headModel)
model.summary()

In [8]:
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['accuracy','Precision','Recall', 'AUC']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [9]:

steps_per_epoch = generator_train.n / batch_size
steps_test = generator_test.n / batch_size

earlystopping = EarlyStopping(monitor ="val_loss", 
                              mode ="min", patience = 3, 
                              restore_best_weights = True)
history = model.fit_generator(generator=generator_train,
                              epochs=50,
                              validation_data=generator_val,
                              validation_steps=steps_test,
                              callbacks =[earlystopping]
                              )


Epoch 1/50
82/82 [==============================] - 106s 963ms/step - loss: 0.8782 - accuracy: 0.6006 - precision: 0.6875 - recall: 0.5030 - auc: 0.7839 - val_loss: 4.9467 - val_accuracy: 0.2658 - val_precision: 0.2658 - val_recall: 0.2658 - val_auc: 0.5127
Epoch 2/50
82/82 [==============================] - 74s 895ms/step - loss: 0.4505 - accuracy: 0.8415 - precision: 0.8678 - recall: 0.7805 - auc: 0.9550 - val_loss: 4.9925 - val_accuracy: 0.2658 - val_precision: 0.2658 - val_recall: 0.2658 - val_auc: 0.5013
Epoch 3/50
82/82 [==============================] - 74s 899ms/step - loss: 0.2813 - accuracy: 0.9085 - precision: 0.9204 - recall: 0.8811 - auc: 0.9864 - val_loss: 3.2298 - val_accuracy: 0.2785 - val_precision: 0.2785 - val_recall: 0.2785 - val_auc: 0.5063
Epoch 4/50
82/82 [==============================] - 74s 901ms/step - loss: 0.1978 - accuracy: 0.9512 - precision: 0.9625 - recall: 0.9390 - auc: 0.9948 - val_loss: 2.3743 - val_accuracy: 0.2532 - val_precision: 0.2532 - val_reca

In [ ]:
headModel = model1.output
headModel = layers.Flatten()(headModel)
model = Model(inputs=model1.input, outputs=headModel)